<h3>輸入原影像，經過壓縮轉換，再由轉換壓縮碼，還原成類原圖</h3>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
from glob import glob
import os
import numpy

In [3]:
import cv2

In [4]:
import tensorflow

In [5]:
from tensorflow import keras

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

from keras import layers
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from pylab import rcParams
from matplotlib import cm

<h3>載入影像檔</h3>

In [7]:
def read_image_file(my_image_file, enhanced=False):
    global max_dim, kernel, alpha, beta
    
    if not os.path.exists(my_image_file):
        return None
        
    kernel = numpy.array([[-1, -1, -1], [-1, 11, -1], [-1, -1, -1]])

    alpha = 0.6 # Contrast control (1.0-3.0)
    beta = 2 # Brightness control (0-100)

    _fish_image = cv2.imread(my_image_file, cv2.COLOR_BGR2RGB)
    if _fish_image is None:
        return None
    
    if enhanced:
        _fish_image = cv2.filter2D(_fish_image, -1, kernel)
        _fish_image = cv2.convertScaleAbs(_fish_image.copy(), alpha=alpha, beta=beta)

    _fish_image = cv2.resize(_fish_image.copy(), max_dim[:2])
    
    return _fish_image

<h3>顯示影像檔</h3>

In [8]:
def show_image(my_image):
    fig, ax = plt.subplots(dpi=120)
    ax.imshow(my_image)
    ax.axis('off')
    plt.show()

<h3>計算原圖與重建差異</h3>

In [10]:
def benchmark(my_diff_image):
    diff = 0.0
    
    for i in range(max_dim[2]):
        diff += numpy.sum(numpy.square(my_diff_image))
        
    return diff**0.5

<h3>產生影像樣本</h3>

In [11]:
def load_data(train_ratio=0.8, enhanced=False):
    global max_dim, image_files

    datagen = ImageDataGenerator(rotation_range=45,
                                 width_shift_range=[0.1, 0.2],
                                 height_shift_range=[0.1, 0.2],
                                 horizontal_flip=True
    )
    
    my_images = []

    for image_file in image_files:
        _image_file = image_file.replace('\\', '/')
        
        _fish_image = read_image_file(_image_file, enhanced)
        if _fish_image is None:
            print('Not Found:', _image_file)
            continue
        
        my_images.append(numpy.reshape(_fish_image, max_dim))

        _fish_img_data = img_to_array(_fish_image)    
        _fish_img_data = numpy.expand_dims(_fish_img_data, 0)
        
        it = datagen.flow(_fish_img_data, batch_size=1)

        for i in range(10):
            _batch = it.next()
            _generated_image = _batch[0].astype('uint8')
            
            my_images.append(numpy.reshape(_generated_image, max_dim))

    my_images = numpy.array(my_images)
    numpy.random.shuffle(my_images)
    
    pos = int(my_images.shape[0] * train_ratio)
    return my_images[:pos], my_images[pos:]

<h3>載入驗證影像檔群</h3>

In [12]:
def load_validate_images(my_faint_image_files):
    global _max_dim
    
    my_faint_images = []

    for image_file in my_faint_image_files:
        _image_file = image_file.replace('\\', '/')

        _fish_image = read_image_file(_image_file, enhance_flag)

        my_faint_images.append(numpy.reshape(_fish_image, max_dim))

    my_faint_images = numpy.array(my_faint_images)

    my_faint_images = my_faint_images.astype('float32') / 255.

    _max_dim = max_dim[0] * max_dim[1] * max_dim[2]
    my_faint_images = numpy.reshape(my_faint_images, (my_faint_images.shape[0], _max_dim))

    print(my_faint_images.shape)
    
    return my_faint_images

<h3>重建驗證影像群</h3>

In [13]:
def predict_validate_images(my_model, my_faint_images):
    faint_encoder = Model(inputs=my_model.input, outputs=my_model.get_layer('latent').output)

    faint_encoded_imgs = faint_encoder.predict(my_faint_images)
    faint_encoded_imgs = faint_encoded_imgs.reshape((len(faint_encoded_imgs), encode_dimension, encode_dimension, 1))

    faint_decoded_imgs = my_model.predict(my_faint_images)

    print(faint_encoded_imgs.shape, faint_decoded_imgs.shape)
    return faint_encoded_imgs, faint_decoded_imgs

<h3>驗證差異影像檔群</h3>

In [14]:
def output_predicted_image(my_faint_image, my_faint_encoded_img, my_faint_decoded_img):
    fig, (ax0, ax1, ax2, ax3) = plt.subplots(dpi=240, ncols=4)

    ax0.imshow(my_faint_image.reshape(max_dim))
    ax0.get_xaxis().set_visible(False)
    ax0.get_yaxis().set_visible(False)

    faint_encoded_img_min = my_faint_encoded_img.min()
    faint_encoded_img_max = my_faint_encoded_img.max()
    faint_encoded_img_val = (my_faint_encoded_img - faint_encoded_img_min) / (faint_encoded_img_max - faint_encoded_img_min)

    ax1.imshow(faint_encoded_img_val.reshape(encode_dimension, encode_dimension), cmap=plt.cm.binary)
    ax1.get_xaxis().set_visible(False)
    ax1.get_yaxis().set_visible(False)

    ax2.imshow(my_faint_decoded_img.reshape(max_dim))
    ax2.get_xaxis().set_visible(False)
    ax2.get_yaxis().set_visible(False)

    faint_diff_image = my_faint_image - my_faint_decoded_img
    faint_diff_image_min = faint_diff_image.min()
    faint_diff_image_max = faint_diff_image.max()
    faint_diff_image_val = (faint_diff_image - faint_diff_image_min) / (faint_diff_image_max - faint_diff_image_min)

    ax3.imshow(faint_diff_image_val.reshape(max_dim), cmap=plt.cm.binary)
    ax3.get_xaxis().set_visible(False)
    ax3.get_yaxis().set_visible(False)

    plt.tight_layout()
    plt.show()

<h3>輸出差異影像檔群</h3>

In [15]:
def output_diff_image(my_faint_image, my_faint_decoded_img):
    global max_dim
    
    fig, ax = plt.subplots(dpi=240)

    faint_diff_image = my_faint_image - my_faint_decoded_img
    faint_diff_image_min = faint_diff_image.min()
    faint_diff_image_max = faint_diff_image.max()
    faint_diff_image_val = (faint_diff_image - faint_diff_image_min) / (faint_diff_image_max - faint_diff_image_min)

    ax.imshow(faint_diff_image_val.reshape(max_dim), cmap=plt.cm.binary)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    plt.show()

<h3>隨機顯示原圖/重建/特徵/差異等影像</h3>

In [16]:
def random_plot_all():
    i = numpy.random.randint(0, faint_decoded_imgs.shape[0], 1)[0]

    output_predicted_image(my_faint_images[i], faint_encoded_imgs[i], faint_decoded_imgs[i])
    benchmark(faint_diff_image_val.reshape(max_dim))

<h3>隨機顯示差異等影像</h3>

In [17]:
def random_plot_diff():
    i = numpy.random.randint(0, faint_decoded_imgs.shape[0], 1)[0]

    output_diff_image(my_faint_images[i], faint_decoded_imgs[i])

<h3>主程式: 環境設定</h3>

In [18]:
image_path = 'images/full'
max_dim = (256, 256, 3)
enhance_flag = False

<h3>蒐集影像檔目錄群</h3>

In [19]:
image_files = [ fn.replace('\\', '/') for fn in glob('%s/F*.jpg' % image_path) ]

_image_ID = numpy.random.randint(0, len(image_files))
print(image_files[_image_ID])

images/full/F007010.jpg


<h3>載入或產生訓練及驗證影像</h3>

In [20]:
mem_file = 'FULL_TRAIN.npy'
x_train = None 

if os.path.exists(mem_file):
    x_train = numpy.load(mem_file)  
    print(x_train.shape)

(20213, 196608)


In [21]:
mem_file = 'FULL_TEST.npy'
x_test = None

if os.path.exists(mem_file):
    x_test = numpy.load(mem_file)
    print(x_test.shape)

(5054, 196608)


In [22]:
if x_train is None or x_test is None:
    x_train, x_test = load_data(train_ratio=0.8, enhanced=enhance_flag)
    
    x_train = x_train.astype('float32') / 255.
    x_test = x_test.astype('float32') / 255.
    
    x_train = x_train.reshape((x_train.shape[0], numpy.prod(x_train.shape[1:])))
    x_test = x_test.reshape((x_test.shape[0], numpy.prod(x_test.shape[1:])))
    
    numpy.save('FULL_TRAIN.npy', x_train)
    numpy.save('FULL_TEST.npy', x_test)

In [23]:
x_train.shape, x_test.shape

((20213, 196608), (5054, 196608))

<h3>參數設置</h3>
<p>$$壓縮率: \frac {65536}{32} = 2048$$</p>

In [24]:
encode_dimension = 12
encoding_dim = encode_dimension * encode_dimension

<h3>載入或建立訓練模型</h3>

In [25]:
model_saved_file = 'models/MLP-C-F-0-128-64-32.hdf5'
autoencoder = None

In [26]:
if not os.path.exists(model_saved_file):
    num_of_epoch = 16
    num_of_batch_size = 8

    input_img = keras.Input(shape=(x_train.shape[1],), name='input')

    encoded_128 = layers.Dense(128, name='encoded_128', activation='relu')(input_img)
    encoded_64 = layers.Dense(64, name='encoded_64', activation='relu')(encoded_128)
    encoded_32 = layers.Dense(32, name='encoded_32', activation='relu')(encoded_64)
    encoded = layers.Dense(encoding_dim, name='latent', activation='relu')(encoded_32)

    decoded_32 = layers.Dense(32, name='decoded_32', activation='relu')(encoded)
    decoded_64 = layers.Dense(64, name='decoded_64', activation='relu')(decoded_32)
    decoded_128 = layers.Dense(128, name='decoded_128', activation='relu')(decoded_64)
    decoded = layers.Dense(x_train.shape[1], name='output', activation='sigmoid')(decoded_128)

    autoencoder = keras.Model(input_img, decoded, name='autoencoder')

    print(autoencoder.summary())

    encoder = keras.Model(input_img, encoded)
    encoded_input = keras.Input(shape=(encoding_dim,))

    decoder_layer_32 = autoencoder.layers[-4]
    decoder_layer_64 = autoencoder.layers[-3]
    decoder_layer_128 = autoencoder.layers[-2]
    decoder_layer = autoencoder.layers[-1]

    decoder = keras.Model(encoded_input, 
                          decoder_layer(decoder_layer_128(
                              decoder_layer_64(
                              decoder_layer_32(encoded_input)))))

    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    checkpointer = ModelCheckpoint(
        filepath=model_aved_file, 
        monitor="val_loss", verbose=1, 
        save_best_only=True, mode='auto'
    )

    earlyStop = EarlyStopping(monitor='val_loss', patience=50)


    score = autoencoder.fit(x_train, x_train,
                            epochs=num_of_epoch,
                            batch_size=num_of_batch_size,
                            shuffle=True,
                            validation_data=(x_test, x_test),
                            callbacks=[checkpointer, earlyStop]
                           )

    best_iteration = numpy.argmin(score.history['val_loss']) + 1

    val_scores = score.history['val_loss'][0:best_iteration]
    train_scores = score.history['loss'][0:best_iteration]

    fig, ax = plt.subplots(dpi=120)

    ax.plot(val_scores, label='val_loss')
    ax.plot(train_scores, label='train_loss')

    plt.legend(loc='upper right')
    plt.show()
else:
    autoencoder = keras.models.load_model(model_saved_file)
    print(autoencoder.summary())

Model: "autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 196608)]          0         
_________________________________________________________________
encoded_128 (Dense)          (None, 128)               25165952  
_________________________________________________________________
encoded_64 (Dense)           (None, 64)                8256      
_________________________________________________________________
encoded_32 (Dense)           (None, 32)                2080      
_________________________________________________________________
latent (Dense)               (None, 144)               4752      
_________________________________________________________________
decoded_32 (Dense)           (None, 32)                4640      
_________________________________________________________________
decoded_64 (Dense)           (None, 64)                

<h3>蒐集驗證影像檔目錄群</h3>

In [27]:
faint_image_path = 'D:/Projects/ComputerVision/ObjectDetection/Fish/source_images'
faint_image_files = sorted(glob('%s/F*.jpg' % faint_image_path))

<h3>影像邊緣偵測參數</h3>

In [28]:
sobel_filter = 7
sobel_scale = 1.2
sobel_delta = 0.2
sobel_weight = 0.75

sobel_ddepth = cv2.CV_16S

<h3>產生驗證影像檔群</h3>

In [39]:
sobel_detect = True

In [40]:
start_position_list = [0, 64, 128, 192, 256]

for j in range(len(start_position_list)):
    start_pos = start_position_list[j]
    
    my_faint_images = load_validate_images(faint_image_files[start_pos:start_pos+64])
    faint_encoded_imgs, faint_decoded_imgs = predict_validate_images(autoencoder, my_faint_images)

    for i in range(my_faint_images.shape[0]):
        my_faint_image = my_faint_images[i]
        my_faint_encoded_img = faint_encoded_imgs[i]
        my_faint_decoded_img = faint_decoded_imgs[i]

        faint_diff_image = my_faint_image - my_faint_decoded_img
        faint_diff_image_min = faint_diff_image.min()
        faint_diff_image_max = faint_diff_image.max()
        faint_diff_image_val = (faint_diff_image - faint_diff_image_min) / (faint_diff_image_max - faint_diff_image_min)
        faint_diff_image_val = faint_diff_image_val.reshape(max_dim)

        output_path = 'DIFF'
        if sobel_detect:
            output_path = 'DIFF2'

            src = cv2.GaussianBlur(faint_diff_image_val, (sobel_filter, sobel_filter), 0)  
            gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)   

            grad_x = cv2.Sobel(gray, sobel_ddepth, 1, 0, ksize=sobel_filter, 
                               scale=sobel_scale, delta=sobel_delta, borderType=cv2.BORDER_DEFAULT)

            grad_y = cv2.Sobel(gray, sobel_ddepth, 0, 1, ksize=sobel_filter, 
                               scale=sobel_scale, delta=sobel_delta, borderType=cv2.BORDER_DEFAULT)

            abs_grad_x = cv2.convertScaleAbs(grad_x)
            abs_grad_y = cv2.convertScaleAbs(grad_y)    

            grad = cv2.addWeighted(abs_grad_x, sobel_weight, abs_grad_y, sobel_weight, 0)
            faint_diff_image_val = grad

        fig, ax = plt.subplots(dpi=300)
        
        ax.axis("off")
        ax.imshow(faint_diff_image_val, cmap=plt.cm.binary)
               
        output_file = '%s/D%06d.jpg' % (output_path, i+start_pos)
        if os.path.exists(output_file):
            os.remove(output_file)
            
        fig.savefig(output_file, bbox_inches='tight',pad_inches=0)
        plt.close()

(64, 196608)
(64, 12, 12, 1) (64, 196608)
(64, 196608)
(64, 12, 12, 1) (64, 196608)
(64, 196608)
(64, 12, 12, 1) (64, 196608)
(64, 196608)
(64, 12, 12, 1) (64, 196608)
(64, 196608)
(64, 12, 12, 1) (64, 196608)
